In [ ]:
import pandas as pd
import re, logging, os, sys, json
import fitz

In [ ]:
def _extract_generic_data(self, main_key: str, data: list, pattern: str):
    final_dict = {}
    for text in data:
        text = re.sub(self.REGEX['escape'], "", text.strip())
        matches = re.findall(self.REGEX[pattern], text, re.IGNORECASE)
        for key, value in matches:
            final_dict[key] = value
    return {main_key: final_dict}

def _extract_str_data(self, key: str, data: list):
    return {key: ' '.join(data)}

def _extract_manager_data(self, main_key: str, data: list, pattern:str):
    final_list = []
    for i in range(0, len(data), 3):
        txt = " ".join(data[i:i+3])
        txt = re.sub(self.REGEX['escape'], "", txt).strip()
        if matches := re.findall(self.REGEX[pattern], txt, re.IGNORECASE):
            for match in matches:
                name, desig, since, exp = match
                final_list.append({
                    "name": name,
                    "designation": desig,
                    "managing_since": since,
                    "experience": exp
                })
    return {main_key: final_list}

def _extract_dum_data(self, key, data):
    return {key: data}

def _extract_scheme_data(self,main_key:str,data:list, pattern:str):
    regex_ = self.REGEX[pattern] #list
    mention_start = regex_[:-1]
    mention_end = regex_[1:]

    patterns = [r"({start})\s*(.+?)\s*({end}|$)".format(start=start, end=end)
        for start, end in zip(mention_start, mention_end)]
    
    final_dict = {}
    scheme_data = " ".join(data)
    for pattern in patterns:
        if matches:= re.findall(pattern, scheme_data, re.DOTALL|re.IGNORECASE):
            for match in matches:
                key, value, dummy = match
                value = value.strip()
                final_dict[key] = value
    
    return {main_key:final_dict}

def match_regex_to_content(self, string: str, data: list, *args):
    for pattern, (func_name, regex_key) in self.PATTERN_TO_FUNCTION.items():
        if re.match(pattern, string, re.IGNORECASE):
            func = getattr(self, func_name)
            if regex_key:
                return func(string, data, regex_key)
            return func(string, data)
    return self._extract_dum_data(string, data)

def _extract_load_data(self,main_key:str,data:list, pattern:str):
        load_data = " ".join(data)
        load_data = re.sub(r"[\^*\$:;~]","",load_data).strip()
        final_dict = {}
        if match:= re.findall(self.REGEX[pattern],load_data.strip(), re.IGNORECASE):
            exit_,entry_ = match[0]
            final_dict['entry_load'] = entry_,
            final_dict['exit_load'] = exit_
        return {main_key:final_dict}

In [ ]:
pattern = r'(?:Entry Load)?\s*(Nil|.*?)\s*Exit Load\s*(.*)$'
for fund, content in final_text.items():
    check = 'scheme_details.load_structure'
    if check in content:
        text =re.sub(r'[^A-Za-z0-9\s\-\(\).,]+', "",content[check]).strip()
        print(text)
        match = re.findall(pattern,text, re.IGNORECASE)
        print(match)
        
pattern = r'(AUM|Monthly Average AUM)\s*([\d,.]+)'
for fund, content in final_text.items():
    check = 'aum'
    if check in content:
        for text in  content[check]:
            text =re.sub(r'[^A-Za-z0-9\s\-\(\).,]+', "",text).strip()
            print(text)
            match = re.findall(pattern,text, re.IGNORECASE)
            print(match)
            
pattern = r'(?:Entry Load)?\s*(Nil|.*?)\s*Exit Load\s*(.*)$'
for fund, content in final_text.items():
    check = 'scheme_details.load_structure'
    if check in content:
        all_text = " ".join(content[check])
        all_text =re.sub(r'[^A-Za-z0-9\s\-\(\).,]+', "", all_text).strip()
        print(all_text)
        print(re.findall(pattern,all_text, re.IGNORECASE))

In [ ]:
min_amt = "([\\d,]+).*?([\\d,]+)"

In [ ]:
mention_start = [
        "Date of Allotment",
        "Fund Manager",
        "Fund Size",
        "Load Structure",
        "Benchmark",
        "Minimum Additional",
        r"Minimum Redemption\s*\/\s*Switch-out Amount",
    ]
    
mention_end = mention_start[1:] + ["End_of_Data"]

# Generate regex patterns dynamically
patterns = [r"({start}\s*)(.+?)({end}|$)".format(start=start, end=end)
    for start, end in zip(mention_start, mention_end)]
final_dict = {}

In [ ]:
def get_unique_keys(json_files):
    unique_keys = set()  # Store unique keys

    for file in json_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)

                for fund_name, fund_data in data.items():
                    if isinstance(fund_data, dict):
                        extract_keys(fund_data, unique_keys)

        except Exception as e:
            print(f"Error reading {file}: {e}")

    return unique_keys

def extract_keys(data, key_set):
    """Recursively extract keys from JSON data."""
    if isinstance(data, dict):
        for key, value in data.items():
            key_set.add("_".join(key.lower().split(" ")))
            extract_keys(value, key_set)
    elif isinstance(data, list):
        for item in data:
            extract_keys(item, key_set)

In [ ]:
json_folder = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output"  # Update this to your folder path
json_files = [os.path.join(json_folder, file) for file in os.listdir(json_folder) if file.endswith('.json')]
unique_keys = get_unique_keys(json_files)


In [ ]:
import re, os, pprint
from app.pdfParse import Reader
from app.fundRegex import FundRegex
import fitz

class FranklinTempleton(Reader):
    PARAMS = {
        'fund': [[25,20],r"^(Franklin|Templeton).*$",[16,24],[-65794]], #[flag], regex_fund_name, range(font_size), [font_color]
        'clip_bbox': [(0,100,180,812)],
        'line_x': 180.0,
        'data': [[6,9],[-16751720],30.0,['ZurichBT-BoldCondensed']], #sizes, color, set_size font_name
        'content_size':[30.0,10.0]
    }
    
    REGEX = {
        'nav': r'(Growth Plan|IDCW Plan|Direct Growth Plan|Direct IDCW Plan)\s*([\d\-,]+\.\d+)',
        'metric': r'(Sharpe Ratio|Standard Deviation|Beta|Annualised)[\s]+([\d\-,]+\.\d+)',
        'expense': r'(Regular|Direct)\s*([\d,.-]+)',
        'load': r'ENTRY LOA(.*?)\s*EXIT LOAD\s*(.*?)$',
        'aum': r'(Month End|Monthly Average)\s*([\d,.\-]+)',
        'ptr':r'^(Portfolio Turnover|Total Portfolio).*\s*([\d\-,]+\.\d+)',
        # 'manager': r'([A-Za-z\s]+)\s*\(Managing Since\s*([A-Za-z0-9\s]+) and overall experience of ([a-z0-9\s]+)\)',
        'escape': r'[^A-Za-z0-9\s\-\(\).,]+'
    }

    PATTERN_TO_FUNCTION = {
        r"^(date|benchmark|investment|type_of|scheme|multiples|minimum).*":("_extract_str_data", None),
        r"^nav.*":("_extract_generic_data", "nav"),
        # r"^fund_mana.*": self.__extract_manager_data,
        r"^aum.*": ("_extract_generic_data", 'aum'),
        r"^portfolio_turnover_ratio": ("_extract_generic_data", 'ptr'),
        r"^load": ("_extract_load_data", 'load'),
        r"^metrics": ("_extract_generic_data", "metric"),
    }

    def __init__(self, paths_config: str):
        super().__init__(paths_config, self.PARAMS)
    
    def _extract_str_data(self, key: str, data: list):
        return {key: ' '.join(data)}
    
    def _extract_dummy_data(self, main_key:str, data:list):
        return{main_key:data}
    
    def _extract_generic_data(self, main_key: str, data: list, pattern: str):
        final_dict = {}
        for text in data:
            text = re.sub(self.REGEX['escape'], "", text.strip())
            matches = re.findall(self.REGEX[pattern], text, re.IGNORECASE)
            for match in matches:
                if len(match) == 2:
                    key, value = match
                    final_dict[key.strip()] = value.strip()
                elif len(match) == 3:
                    key,v1,v2 = match
                    final_dict[key.strip()] = v1.strip()
        return {main_key: final_dict}
    
    def _extract_manager_data(self, main_key: str, data: list, pattern:str):
        manager_data = " ".join(data)
        manager_data = re.sub(self.REGEX[pattern], "", manager_data).strip()
        final_list = []
        if matches := re.findall(self.REGEX['manager'], manager_data, re.IGNORECASE):
            for match in matches:
                name, since, exp = match
                final_list.append({
                    "name": name.strip(),
                    "designation": "",
                    "managing_since": since.strip(),
                    "experience": exp.strip()
                })
        return {main_key: final_list}
    
    def _extract_load_data(self,main_key:str,data:list, pattern:str):
        load_data = " ".join(data)
        load_data = re.sub(self.REGEX['escape'], "", load_data).strip()
        final_dict = {}
        if match:= re.findall(self.REGEX[pattern],load_data.strip(), re.IGNORECASE):
            exit_,entry_ = match[0]
            final_dict['entry_load'] = entry_,
            final_dict['exit_load'] = exit_
        return {main_key:final_dict}
        
    
    def match_regex_to_content(self, string: str, data: list, *args):
        for pattern, (func_name, regex_key) in self.PATTERN_TO_FUNCTION.items():
            if re.match(pattern, string, re.IGNORECASE):
                func = getattr(self, func_name)
                if regex_key and regex_key in self.REGEX:
                    return func(string, data, regex_key)
                return func(string, data)
        return self._extract_dummy_data(string, data)


IMPORTANT CODE DONT DELETE

In [ ]:


""" thissssssssssssssssssssssss"""
class HSBC(Reader):
    
    PARAMS = {
        'fund': [[20,16], r'^(HSBC|Bharat).*Fund$',[12,20],[-1237724]],
        'clip_bbox': [(0,100,180,812)],
        'line_x': 180.0,
        'data': [[6,8], [-16777216], 30.0, ['Arial-BoldMT']],
        'content_size':[30.0,10.0]
    }
    
    def __init__(self,paths_config:str):
        super().__init__(paths_config, self.PARAMS)
        
    #Fund REGEX
    def __return_all_data(self,key,data:list):
        return {key:data}
    
    def __extract_inv_data(self,key:str, data:list):            
        return {key: ' '.join(data)}
    
    #MAPPING
    def match_regex_to_content(self, string:str, data:list):
        check_header = string
        if re.match(r"^(investment).*", check_header, re.IGNORECASE):
            return self.__extract_inv_data(string,data)
        # elif re.match(r"^total.*", check_header, re.IGNORECASE):
        #     return self.__extract_total_expense_data(string, data)
        # elif re.match(r"^nav.*", check_header, re.IGNORECASE):
        #     return self.__extract_nav_data(string, data)
        # elif re.match(r"^portfolio.*", check_header, re.IGNORECASE): #error here portfolio is read
        #     return self.__extract_scheme_data(string, data)
        return self.__return_all_data(string,data)     


class PPFAIS(Reader):
    PARAMS = {
        'fund': [[16,20], r'^(Parag).*',[14,24],[-16777216,-13159371,-14869475]],
        'clip_bbox': [(0,65,290,812)],
        'line_x': 290.0,
        'data': [[6,10], [-65794], 30.0, ['Arial-BoldMT',]]
        }
    
    def __init__(self, paths_config:str):
        super().__init__(paths_config,self.PARAMS)
        
    #REGEX
    def __return_all_data(self,main_key,data:list):
        return {main_key:data}
    
    def __extract_inv_data(self,main_key:str, data:list):            
        return {main_key: ' '.join(data)}
    
    #MAPPING
    def match_regex_to_content(self, string: str, data: list):
        pattern_to_function = {
            r"^investment.*": self.__extract_inv_data,
            # r"^scheme.*": self.__extract_scheme_data,
            # r"^metrics.*": self.__extract_metric_data,
        }

        for pattern, func in pattern_to_function.items():
            if re.match(pattern, string, re.IGNORECASE):
                return func(string, data)

        return self.__return_all_data(string, data)


In [ ]:
#first
final_dic = {
    "metadata": [],
    "records": []
}

#second
meta_data = {
    "document_name": "",
    "file_type": "",
    "process_date": ""
}
record = {
            "amc_name": "",
            "benchmark_index":[],
            "field_location": [],
            "fund_manager": [],
            "load": [],
            "main_scheme_name": "",
            "metrics": [],
            "min_addl_amt": "",
            "min_addl_multiple":"",
            "min_amt":"",
            "monthly_aaum_date":"",
            "monthly_aaum_value":"",
            "mutual_fund_name": "",
            "scheme_launch_date":""
            
        }

#third  
fund_manager = {
    "managing_fund_since":"",
    "name": "",
    "qualification": "",
    "total_exp": ""
}
load = {
    "type": "",
    "comment": ""
}   
metrics= {
    "name":"",
    "value":""
}    
field_location = {
    "amc_name": "",
    "benchmark_index": "",
    "count": 0,
    "fund_manager_managing_fund_since": "",
    "fund_manager_name": "",
    "fund_manager_total_exp": "",
    "load_entry": "",
    "load_exit": "",
    "main_scheme_name": "",
    "metrics_beta": "",
    "metrics_port_turnover_ratio": "",
    "metrics_r_squared_ratio": "",
    "metrics_sharpe": "",
    "metrics_std_dev": "",
    "metrics_treynor_ratio": "",
    "min_addl_amt": "",
    "min_addl_amt_multiple": "",
    "min_amt": "",
    "min_amt_multiple": "",
    "monthly_aaum_date": "",
    "monthly_aaum_value": "",
    "mutual_fund_name": "",
    "scheme_launch_date": ""
    }

In [ ]:
# Aditya Birla Sun Life Mutual Fund
# Kotak Mahindra Mutual Fund
# Shriram Mutual fund
# PPFAS Mutual Fund
# ITI Mutual Fund
# PGIM India Mutual Fund
# BajaJ Finserv Mutual Fund
# Union Mutual Fund
# Sundaram Mutual Fund
# NJ Mutual Fund
# Samco Mutual Fund
# Bank of India Mutual Fund
# DSP Mutual Fund
# Edelweiss Mutual Fund
# Invesco Mutual Fund
# Helios Mutual Fund
# Groww Mutual Fund
# Nippon India Mutual Fund
# Quantum Mutual Fund
# Franklin Templeton Mutual Fund
# Bandhan Mutual Fund
# IL&FS Mutual Fund (IDF)
# Tata Mutual Fund
# Mirae Asset Mutual Fund
# Motilal Oswal Mutual Fund
# Old Bridge Mutual Fund
# 360 ONE Mutual Fund
# LIC Mutual Fund
# Mahindra Manulife Mutual Fund
# WhiteOak Mutual Fund
# Navi Mutual Fund
# Baroda BNP Paribas Mutual Fund
# SBI Mutual Fund
# Taurus Mutual Fund
# JM Financial Mutual Fund
# Trust Mutual Fund
# Zerodha Mutual Fund
# ICICI Prudential Mutual Fund
# Quant Mutual Fund
# Axis Mutual Fund
# Canara Robeco Mutual Fund
# HDFC Mutual Fund
# HSBC Mutual Fund
# UTI Mutual Fund
# Angel One Mutual Fund
# Unifi Mutual Fund

In [ ]:
# Axis Asset Management Company Limited
# Aditya Birla Sun Life AMC Limited
# Bank of India Investment Managers Private Limited
# Canara Robeco Asset Management Company Limited
# PGIM India Asset Management Private Limited
# DSP Investment Managers Private Limited
# Edelweiss Asset Management Limited
# Quant Money Managers Limited
# Franklin Templeton Asset Management (India) Private Limited
# HDFC Asset Management Company Limited
# HSBC Asset Management (India) Private Limited
# ICICI Prudential Asset Management Company Limited
# Bandhan AMC Limited
# 360 ONE Asset Management Limited
# IL&FS Infra Asset Management Limited
# Groww Asset Management Limited
# Invesco Asset Management (India) Private Limited
# JM Financial Asset Management Limited
# Kotak Mahindra Asset Management Company Limited(KMAMCL)
# LIC Mutual Fund Asset Management Limited
# Mahindra Manulife Investment Management Private Limited
# Mirae Asset Investment Managers (India) Private Limited
# Motilal Oswal Asset Management Company Limited
# PPFAS Asset Management Private Limited
# Quantum Asset Management Company Private Limited
# Nippon Life India Asset Management Limited
# SBI Funds Management Limited
# Shriram Asset Management Company Limited
# Sundaram Asset Management Company Limited
# Tata Asset Management Private Limited
# Taurus Asset Management Company Limited
# Union Asset Management Company Private Limited
# UTI Asset Management Company Limited
# WhiteOak Capital Asset Management Limited
# ITI Asset Management Limited
# Navi AMC Limited
# NJ Asset Management Private Limited
# Samco Asset Management Private Limited
# Trust Asset Management Private Limited
# Baroda BNP Paribas Asset Management India Private Limited
# Bajaj Finserv Asset Management Limited
# Helios Capital Asset Management (India) Private Limited
# Zerodha Asset Management Private Limited
# Old Bridge Asset Management Private Limited
# Angel One Asset Management Company Limited
# Unifi Asset Management Private Limited

In [ ]:
def _update_date_data(self,main_key:str,data):
    dates = {
        'pgim india large cap fund': '30/01/2003|01/01/2013',
        'pgim india flexi cap fund': '04/08/2015|04/03/2015',
        'pgim india large and mid cap fund': '12/02/2024|12/02/2024',
        'pgim india multicap fund': '10/09/2024|10/09/2024',
        'pgim india midcap opportunities fund': '02/12/2013|02/12/2013',
        'pgim india small cap fund': ' 29/07/2021| 29/07/2021',
        'pgim india t elss tax saver fund': '11/12/2015|11/12/2015',
        'pgim india healthcare fund': '06/12/2024|06/12/2024',
        'pgim india retirement fund': '15/04/2024|15/04/2024',
        'pgim india emerging markets equity fund': '11/09/2007|01/01/2013',
        'pgim india global equity opportunities fund': '14/05/2010|01/01/2013',
        'pgim india global select real estate securities fund of fund': '03/12/2021|03/12/2021',
        'pgim india hybrid equity fund': '05/02/2004| 01/01/2013',
        'pgim india d arbitrage fund': '27/08/2014|27/08/2014',
        'pgim india d equity savings fund': '05/02/2004|01/01/2013',
        'pgim india balanced advantage fund': '04/02/2021|04/02/2021',
        'pgim india overnight fund': '27/08/2019|27/08/2019',
        'pgim india liquid fund': ': 05/09/2007|01/01/2013',
        'pgim india ultra short duration fun': '14/07/2008|01/01/2013',
        'pgim india money market fund': '06/03/2020|06/03/2020',
        'pgim india t dynamic bond fund': '12/01/2012|01/01/2013',
        'pgim india corporate bond fund': '30/01/2003| 01/01/2013 ',
        'pgim india gilt fund': '27/10/2008|01/01/2013',
        'pgim india crisil ibx gilt index - apr 2028 fund': '22/02/2023|22/02/2023'
    }

    pass

In [ ]:
#finkstein

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="cog_updated_db"
)

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM mf_amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO mf_amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid

def insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month,data_from):
    keys = ["amc_id", "entered_time", "amc_for_month","data_from", "amc_name", "benchmark_index",
            "main_scheme_name", "mutual_fund_name", "monthly_aaum_date", "monthly_aaum_value",
            "scheme_launch_date", "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple"]
    
    query = f"""
        INSERT INTO mf_mutual_funds ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    values = [
        amc_id, curr_time, amc_month,data_from, details.get("amc_name", ""),
        ", ".join(details.get("benchmark_index", [])),
        details.get("main_scheme_name", ""),
        details.get("mutual_fund_name", ""),
        details.get("monthly_aaum_date", ""),
        details.get("monthly_aaum_value", ""),
        details.get("scheme_launch_date", ""),
        details.get("min_addl_amt", ""),
        details.get("min_addl_amt_multiple", ""),
        details.get("min_amt", ""),
        details.get("min_amt_multiple", "")
    ]

    try:
        cursor.execute(query, values)
    except Exception as e:
        print("Error inserting mutual fund:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Query:", query)
        print("Values:", values)
        print("Exception:", e)
        return None
    return cursor.lastrowid

def insert_fund_managers(cursor, amc_id, entered_time, amc_for_month, data_from, mutual_fund_id, details):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_fund_managers
            (amc_id, entered_time, amc_for_month, data_from, mutual_fund_id, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            amc_id,
            entered_time,
            amc_for_month,
            data_from,
            mutual_fund_id,
            details["main_scheme_name"],
            manager.get("name", ""),
            manager.get("qualification", ""),
            manager.get("managing_fund_since", ""),
            manager.get("total_exp", "")
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error in Manager: {e}")

def insert_load(cursor, amc_id, mutual_fund_id, details):
    entry_load = exit_load = ""
    for item in details.get("load", []):
        if item.get("type") == "entry":
            entry_load = item.get("comment", "")
        elif item.get("type") == "exit":
            exit_load = item.get("comment", "")

    query = """
        INSERT INTO mf_transformed_loads
        (amc_id, mutual_fund_id, main_scheme_name, entry_load, exit_load)
        VALUES (%s, %s, %s, %s, %s)
    """
    values = [
        amc_id,
        mutual_fund_id,
        details.get("main_scheme_name", ""),
        entry_load,
        exit_load
    ]
    cursor.execute(query, values)

def insert_metrics(cursor, amc_id, mutual_fund_id, details):
    metric_template = {
        "alpha": "", "arithmetic_mean_ratio": "", "average_div_yld": "", "average_pb": "", "average_pe": "",
        "avg_maturity": "", "beta": "", "correlation_ratio": "", "downside_deviation": "", "information_ratio": "",
        "macaulay": "", "mod_duration": "", "port_turnover_ratio": "", "r_squared_ratio": "", "roe_ratio": "",
        "sharpe": "", "sortino_ratio": "", "std_dev": "", "tracking_error": "", "treynor_ratio": "",
        "upside_deviation": "", "ytm": ""
    }

    for metric in details.get("metrics", []):
        metric_name = metric.get("name")
        if metric_name in metric_template:
            metric_template[metric_name] = metric.get("value", "")

    keys = ["amc_id", "main_scheme_name", "mutual_fund_id"] + list(metric_template.keys())
    query = f"""
        INSERT INTO mf_transformed_metrics
        ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """
    values = [
        amc_id,
        details.get("main_scheme_name", ""),
        mutual_fund_id
    ] + list(metric_template.values())

    cursor.execute(query, values)

# Begin inserting all records
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB 25"
data_from = "finkstein"

for amc_key, records in combined_records.items():
    print(f"Doing {amc_key}")
    for record in records:
        try:
            details = record["value"]
            amc_id = get_or_create_amc(details["amc_name"])
            mutual_fund_id = insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_managers(cursor, amc_id, curr_time, amc_month, data_from, mutual_fund_id, details)
                insert_load(cursor, amc_id, mutual_fund_id, details)
                insert_metrics(cursor, amc_id, mutual_fund_id, details)
            else:
                print
        except Exception as e:
            print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")
    print(f"Done {amc_key}")
conn.commit()
cursor.close()
conn.close()
print("Data insertion completed.")


In [ ]:
# mydata
# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="1234",
#     database="cog_updated_db"
# )

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="try_db"
)

# conn = mysql.connector.connect(
#     host="172.22.225.155",
#     user="cog_mf",
#     password="bnYwFChjLAV2Z%9E",
#     database="cog_mf",
#     port=3306,
#     charset='utf8mb4'
# )
cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM mf_amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO mf_amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid

def insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month, data_from):
    keys = ["amc_id", "entered_time", "amc_for_month", "data_from", "amc_name", "benchmark_index",
            "main_scheme_name", "mutual_fund_name", "monthly_aaum_date", "monthly_aaum_value",
            "scheme_launch_date", "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple"]

    query = f"INSERT INTO mf_mutual_funds ({', '.join(keys)}) VALUES ({', '.join(['%s'] * len(keys))})"
    values = [
        amc_id, curr_time, amc_month, data_from, details.get("amc_name", ""),
        details.get("benchmark_index", ""),
        details.get("main_scheme_name", ""),
        details.get("mutual_fund_name", ""),
        details.get("monthly_aaum_date", ""),
        details.get("monthly_aaum_value", ""),
        details.get("scheme_launch_date", ""),
        details.get("min_addl_amt", ""),
        details.get("min_addl_amt_multiple", ""),
        details.get("min_amt", ""),
        details.get("min_amt_multiple", "")
    ]
    try:
        cursor.execute(query, values)
    except Exception as e:
        print("Error inserting mutual fund:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Query:", query)
        print("Values:", values)
        print("Exception:", e)
        return None
    
    return cursor.lastrowid

def insert_fund_managers(cursor, amc_id,entered_time, amc_for_month,data_from, mutual_fund_id, details):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_fund_managers
            (amc_id,entered_time, amc_for_month, data_from, mutual_fund_id, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            amc_id,
            entered_time, 
            amc_for_month,
            data_from,
            mutual_fund_id,
            details["main_scheme_name"],
            manager.get("name", ""),
            manager.get("qualification", ""),
            manager.get("managing_fund_since", ""),
            manager.get("total_exp", "")
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error in Manager: {e}")

def insert_load(cursor, amc_id, mutual_fund_id, details):
    load = details.get("load", {})
    if not isinstance(load, dict):
        print(f"Error in Loads, {details.get('main_scheme_name', '')}")
        return
    query = """
        INSERT INTO mf_transformed_loads
        (amc_id, mutual_fund_id, main_scheme_name, entry_load, exit_load)
        VALUES (%s, %s, %s, %s, %s)
    """
    values = [
        amc_id,
        mutual_fund_id,
        details.get("main_scheme_name", ""),
        load.get("entry_load", ""),
        load.get("exit_load", "")
    ]
    
    try:
        cursor.execute(query, values)
    except Exception as e:
        print(f"Error in Load: {e}")

def insert_metrics(cursor, amc_id, mutual_fund_id, details):
    keys = ['amc_id', 'main_scheme_name', 'mutual_fund_id', "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity", 
            "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay", "mod_duration", "port_turnover_ratio", "r_squared_ratio", 
            "roe_ratio", "sharpe", "sortino_ratio", "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"]
    metrics = details.get("metrics", {})
    if not isinstance(metrics, dict):
        print(f"Error in Metrics, {details.get('main_scheme_name', '')}")
        return
    query = f"INSERT INTO mf_transformed_metrics ({', '.join(keys)}) VALUES ({', '.join(['%s'] * len(keys))})"
    values = [amc_id, details.get("main_scheme_name", ""), mutual_fund_id] + [metrics.get(k, "") for k in keys[3:]]
    try:
        cursor.execute(query, values)
    except Exception as e:
        print(f"Error in Metrics: {e}")

curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEBRUARY 25"
data_from = "mydata"

for amc_name, schemes in combined_records.items():
    print(f"Inserting Data For: {amc_name}")
    amc_id = get_or_create_amc(next(iter(schemes.values())).get("amc_name", ""))
    for scheme_name, details in schemes.items():
        try:
            mutual_fund_id = insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_managers(cursor, amc_id,curr_time, amc_month, data_from, mutual_fund_id, details)
                insert_load(cursor, amc_id, mutual_fund_id, details)
                insert_metrics(cursor, amc_id, mutual_fund_id, details)
            else:
                print(f"Skipped Data Insertion for AMC {scheme_name}")
        except Exception as e:
            print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")

conn.commit()
cursor.close()
conn.close()
print("Working!!")


In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime
import os
import re

def sanitize_sheet_name(name):
    name = re.sub(r'[^A-Za-z0-9_ ]+', '', name)
    return name.strip().replace(" ", "_")[:31]

def fetch_data_for_source(conn, amc_name, source, entered_time_pattern="%"):
    query = f"""
    SELECT
        mf.data_from,mf.entered_time,mf.amc_for_month,mf.main_scheme_name,mf.benchmark_index,mf.monthly_aaum_value,
        mf.monthly_aaum_value,mf.scheme_launch_date,mf.min_addl_amt,mf.min_addl_amt_multiple,mf.min_amt,mf.min_amt_multiple,tl.entry_load, tl.exit_load,
        tm.alpha,tm.arithmetic_mean_ratio,tm.average_div_yld,tm.average_pb,tm.average_pe,tm.avg_maturity,tm.beta,tm.correlation_ratio,tm.downside_deviation,
        tm.information_ratio,tm.macaulay,tm.mod_duration,tm.port_turnover_ratio,tm.r_squared_ratio, tm.roe_ratio, tm.sharpe,tm.sortino_ratio,tm.std_dev,
        tm.tracking_error,tm.treynor_ratio,tm.upside_deviation,tm.ytm
    FROM mf_mutual_funds mf
    LEFT JOIN mf_transformed_loads tl ON mf.id = tl.mutual_fund_id
    LEFT JOIN mf_transformed_metrics tm ON mf.id = tm.mutual_fund_id
    WHERE mf.amc_name LIKE %s AND mf.data_from = %s AND mf.entered_time LIKE %s;
    """
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query, (f"%{amc_name}%", source, entered_time_pattern))
    rows = cursor.fetchall()
    cursor.close()
    return pd.DataFrame(rows) if rows else pd.DataFrame()

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="1234",
#     database="cog_updated_db"
# )

# cursor = conn.cursor()

amc_names = pd.read_sql("SELECT amc_name FROM mf_amcs;", conn)["amc_name"].tolist()

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_path = f"comparison_export_{timestamp}.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    workbook = writer.book
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1
    })
    gray_format = workbook.add_format({'bg_color': '#D3D3D3'})  # for mydata rows

    for amc in amc_names:
        df_A = fetch_data_for_source(conn, amc, "finkstein", "%2025-04-23%")
        df_B = fetch_data_for_source(conn, amc, "mydata", "%2025-04-23%")

        if df_A.empty and df_B.empty:
            print(f"Skipped: {amc} (no data)")
            continue

        df_A["data_from"] = "finkstein"
        df_B["data_from"] = "mydata"

        combined_df = pd.concat([df_A, df_B], ignore_index=True)
        # combined_df = combined_df.sort_values(by="main_scheme_name")
        front_cols = ['data_from', 'main_scheme_name']
        remaining_cols = [col for col in combined_df.columns if col not in front_cols]
        combined_df = combined_df[front_cols + remaining_cols]

        sheet_name = sanitize_sheet_name(amc)
        combined_df.to_excel(writer, sheet_name=sheet_name, index=False)
        worksheet = writer.sheets[sheet_name]

        for col_num, header in enumerate(combined_df.columns.values):
            worksheet.write(0, col_num, header, header_format)
            header_width = len(header)
            worksheet.set_column(col_num, col_num, header_width + 8)


        # Highlight mydata rows
        for row_num, row in enumerate(combined_df["data_from"], start=1):
            if row == "mydata":
                worksheet.set_row(row_num, None, gray_format)

print(f"Data exported to: {output_path}")
conn.close()
